In [15]:
import sys
import os

# Dynamically add SEAL-Python directory to the Python path
# Must follow the steps in README.md to generate Seal-Python directory for library usage
seal_python_path = os.path.join(os.getcwd(), 'SEAL-Python')
sys.path.append(seal_python_path)

from seal import *
import pandas as pd
import numpy as np
import time  # For measuring time

def example_ckks_credit_scores(csv_path):
    # Load the dataset
    data = pd.read_csv(csv_path)

    # Extract columns
    income = data['INCOME'].tolist()
    savings = data['SAVINGS'].tolist()
    debt = data['DEBT'].tolist()

    # Set up encryption parameters for CKKS
    parms = EncryptionParameters(scheme_type.ckks)
    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 60]))
    scale = pow(2.0, 40)
    context = SEALContext(parms)

    # Generate keys
    keygen = KeyGenerator(context)
    secret_key = keygen.secret_key()
    public_key = keygen.create_public_key()

    # Set up encryptor, evaluator, decryptor, and encoder
    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)
    encoder = CKKSEncoder(context)

    # Encode and encrypt data
    income_plain = encoder.encode(income, scale)
    income_encrypted = encryptor.encrypt(income_plain)

    # Measure time for homomorphic sum and average
    start_homomorphic = time.time()

    # Compute sums homomorphically
    income_sum_encrypted = Ciphertext(income_encrypted)
    for i in range(1, len(income)):
        temp_plain = encoder.encode([income[i]], scale)
        temp_encrypted = encryptor.encrypt(temp_plain)
        evaluator.add_inplace(income_sum_encrypted, temp_encrypted)

    # Decrypt and compute average in plaintext
    income_sum_plain = decryptor.decrypt(income_sum_encrypted)
    income_sum_result = encoder.decode(income_sum_plain)
    homomorphic_avg = income_sum_result[0] / len(income)

    end_homomorphic = time.time()

    # Measure time for plaintext sum and average
    start_plaintext = time.time()
    plaintext_avg = sum(income) / len(income)
    end_plaintext = time.time()

    # Print results and timings
    print(f"Homomorphic Average (Income): {homomorphic_avg:.2f}")
    print(f"Plaintext Average (Income): {plaintext_avg:.2f}")

    print(f"Time for Homomorphic Sum and Average: {end_homomorphic - start_homomorphic:.4f} seconds")
    print(f"Time for Plaintext Sum and Average: {end_plaintext - start_plaintext:.4f} seconds")



if __name__ == "__main__":
    csv_path = 'credit_score_data.csv' 
    example_ckks_credit_scores(csv_path)


Homomorphic Average (Income): 121610.02
Plaintext Average (Income): 121610.02
Time for Homomorphic Sum and Average: 2.8940 seconds
Time for Plaintext Sum and Average: 0.0000 seconds


In [16]:
import sys
import os
import time
from concurrent.futures import ThreadPoolExecutor
from seal import *
import pandas as pd
import numpy as np

# Dynamically add SEAL-Python directory to the Python path
seal_python_path = os.path.join(os.getcwd(), 'SEAL-Python')
sys.path.append(seal_python_path)

def process_chunk(income_chunk, scale, context, encryptor, evaluator, encoder, decryptor):
    """Process a chunk of data (simulating one 'node')"""
    # Encode and encrypt the chunk of income data
    income_plain = encoder.encode(income_chunk, scale)
    income_encrypted = encryptor.encrypt(income_plain)

    # Compute sums homomorphically for the chunk
    income_sum_encrypted = Ciphertext(income_encrypted)
    for i in range(1, len(income_chunk)):
        temp_plain = encoder.encode([income_chunk[i]], scale)
        temp_encrypted = encryptor.encrypt(temp_plain)
        evaluator.add_inplace(income_sum_encrypted, temp_encrypted)

    # Decrypt the result and return the sum
    income_sum_plain = decryptor.decrypt(income_sum_encrypted)
    income_sum_result = encoder.decode(income_sum_plain)
    return income_sum_result[0]

def example_ckks_credit_scores_parallel(csv_path):
    # Load the dataset
    data = pd.read_csv(csv_path)

    # Extract columns
    income = data['INCOME'].tolist()

    # Set up encryption parameters for CKKS
    parms = EncryptionParameters(scheme_type.ckks)
    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 60]))
    scale = pow(2.0, 40)
    context = SEALContext(parms)

    # Generate keys
    keygen = KeyGenerator(context)
    secret_key = keygen.secret_key()
    public_key = keygen.create_public_key()

    # Set up encryptor, evaluator, decryptor, and encoder
    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)
    encoder = CKKSEncoder(context)

    # Split the data into 4 chunks for 4 parallel "nodes"
    chunk_size = len(income) // 4
    income_chunks = [income[i:i + chunk_size] for i in range(0, len(income), chunk_size)]

    # Measure time for parallel processing
    start_parallel = time.time()

    # Use ThreadPoolExecutor to simulate parallel processing on 4 nodes
    with ThreadPoolExecutor(max_workers=8) as executor:
        results = list(executor.map(
            process_chunk,
            income_chunks,
            [scale] * 4,
            [context] * 4,
            [encryptor] * 4,
            [evaluator] * 4,
            [encoder] * 4,
            [decryptor] * 4
        ))

    # Aggregate the results
    total_sum = sum(results)
    homomorphic_avg = total_sum / len(income)

    end_parallel = time.time()

    # Measure time for plaintext sum and average
    start_plaintext = time.time()
    plaintext_avg = sum(income) / len(income)
    end_plaintext = time.time()

    # Print results and timings
    print(f"Homomorphic Average (Income): {homomorphic_avg:.2f}")
    print(f"Plaintext Average (Income): {plaintext_avg:.2f}")
    print(f"Time for Parallel Homomorphic Sum and Average: {end_parallel - start_parallel:.4f} seconds")
    print(f"Time for Plaintext Sum and Average: {end_plaintext - start_plaintext:.4f} seconds")

if __name__ == "__main__":
    csv_path = 'credit_score_data.csv'
    example_ckks_credit_scores_parallel(csv_path)


Homomorphic Average (Income): 121610.02
Plaintext Average (Income): 121610.02
Time for Parallel Homomorphic Sum and Average: 2.8781 seconds
Time for Plaintext Sum and Average: 0.0000 seconds


In [13]:
import sys
import os
import math
import time  # For measuring time

# Dynamically add SEAL-Python directory to the Python path
seal_python_path = os.path.join(os.getcwd(), 'SEAL-Python')
sys.path.append(seal_python_path)

from seal import *
import pandas as pd
import numpy as np

def example_ckks_credit_scores(csv_path):
    # Load the dataset
    data = pd.read_csv(csv_path)

    # Extract columns
    income = data['INCOME'].tolist()

    # Set up encryption parameters for CKKS
    parms = EncryptionParameters(scheme_type.ckks)
    poly_modulus_degree = 8192
    parms.set_poly_modulus_degree(poly_modulus_degree)
    parms.set_coeff_modulus(CoeffModulus.Create(poly_modulus_degree, [60, 40, 40, 60]))
    scale = pow(2.0, 40)
    context = SEALContext(parms)

    # Generate keys
    keygen = KeyGenerator(context)
    secret_key = keygen.secret_key()
    public_key = keygen.create_public_key()

    # Set up encryptor, evaluator, decryptor, and encoder
    encryptor = Encryptor(context, public_key)
    evaluator = Evaluator(context)
    decryptor = Decryptor(context, secret_key)
    encoder = CKKSEncoder(context)

    # Encode and encrypt data
    income_plain = encoder.encode(income, scale)
    income_encrypted = encryptor.encrypt(income_plain)

    # Measure time for homomorphic standard deviation calculation
    start_homomorphic = time.time()

    # Compute the squared sum homomorphically (for variance)
    squared_sum_encrypted = Ciphertext(income_encrypted)
    for i in range(1, len(income)):
        temp_plain = encoder.encode([income[i]], scale)
        temp_encrypted = encryptor.encrypt(temp_plain)
        evaluator.add_inplace(squared_sum_encrypted, temp_encrypted)

    # Compute squared values for variance
    income_squared = [x**2 for x in income]
    income_squared_plain = encoder.encode(income_squared, scale)
    income_squared_encrypted = encryptor.encrypt(income_squared_plain)

    # Compute the squared sum of squares homomorphically
    squared_sum_of_squares_encrypted = Ciphertext(income_squared_encrypted)
    for i in range(1, len(income_squared)):
        temp_plain = encoder.encode([income_squared[i]], scale)
        temp_encrypted = encryptor.encrypt(temp_plain)
        evaluator.add_inplace(squared_sum_of_squares_encrypted, temp_encrypted)

    # Decrypt squared sum and squared sum of squares
    squared_sum_plain = decryptor.decrypt(squared_sum_encrypted)
    squared_sum_of_squares_plain = decryptor.decrypt(squared_sum_of_squares_encrypted)

    # Decode results and compute variance and standard deviation in plaintext
    squared_sum_result = encoder.decode(squared_sum_plain)
    squared_sum_of_squares_result = encoder.decode(squared_sum_of_squares_plain)

    variance_homomorphic = (squared_sum_of_squares_result[0] / len(income)) - (squared_sum_result[0] / len(income)) ** 2
    stddev_homomorphic = math.sqrt(variance_homomorphic)

    end_homomorphic = time.time()

    # Measure time for plaintext standard deviation calculation
    start_plaintext = time.time()

    mean_plain = sum(income) / len(income)
    variance_plain = sum((x - mean_plain) ** 2 for x in income) / len(income)
    stddev_plain = math.sqrt(variance_plain)

    end_plaintext = time.time()

    # Print results and timings
    print(f"Homomorphic Standard Deviation (Income): {stddev_homomorphic:.2f}")
    print(f"Plaintext Standard Deviation (Income): {stddev_plain:.2f}")

    print(f"Time for Homomorphic Standard Deviation Calculation: {end_homomorphic - start_homomorphic:.4f} seconds")
    print(f"Time for Plaintext Standard Deviation Calculation: {end_plaintext - start_plaintext:.4f} seconds")



if __name__ == "__main__":
    csv_path = 'credit_score_data.csv' 
    example_ckks_credit_scores(csv_path)

Homomorphic Standard Deviation (Income): 113659.83
Plaintext Standard Deviation (Income): 113659.83
Time for Homomorphic Standard Deviation Calculation: 5.7529 seconds
Time for Plaintext Standard Deviation Calculation: 0.0001 seconds
